# 6.5 Task 6.5 SRS derivation

## Table of Contents

#### 01. Import libraries and data

#### 02. Derive country SRS scores

#### 03. Key project finding

### 01. Import libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [3]:
# Import data
path = r'C:\Users\dirk8\CareerFoundry Projects\05-2025 Exogenous Shocks Analysis\Data\Prepared Data'
df = pd.read_excel(os.path.join(path, 'SRS Component Data - Rankings.xlsx'), index_col=None)

### 02. Derive country SRS ("Shock Resilience Score")

In [6]:
# Check dimensions of imported file
df.shape

(7, 18)

In [8]:
# Check datatypes
df.dtypes

country                     object
pc1_dis_gfc_rank             int64
pc1_dis_cpc_rank             int64
pc1_dis_cov19_rank           int64
pc1_rec_gfc_rank             int64
pc1_rec_cpc_rank             int64
pc1_rec_cov19_rank           int64
pc2_dis_gfc_rank             int64
pc2_dis_cpc_rank             int64
pc2_dis_cov19_rank           int64
pc2_rec_gfc_rank             int64
pc2_rec_cpc_rank             int64
pc2_rec_cov19_rank           int64
cluster_shifts_flag_rank     int64
policy_lag_gfc_rank          int64
policy_lag_cpc_rank          int64
policy_lag_cov19_rank        int64
missing_variables_rank       int64
dtype: object

In [10]:
# Check contents
df

,country,pc1_dis_gfc_rank,pc1_dis_cpc_rank,pc1_dis_cov19_rank,pc1_rec_gfc_rank,pc1_rec_cpc_rank,pc1_rec_cov19_rank,pc2_dis_gfc_rank,pc2_dis_cpc_rank,pc2_dis_cov19_rank,pc2_rec_gfc_rank,pc2_rec_cpc_rank,pc2_rec_cov19_rank,cluster_shifts_flag_rank,policy_lag_gfc_rank,policy_lag_cpc_rank,policy_lag_cov19_rank,missing_variables_rank
0,Norway,5,7,4,3,6,1,2,5,2,4,5,1,1,3,1,5,1
1,United States,6,1,7,5,1,6,7,1,7,7,1,6,4,6,7,6,1
2,Germany,3,6,1,4,7,5,5,3,6,6,7,7,5,3,1,4,1
3,Japan,7,1,6,6,1,7,1,4,3,1,4,5,2,6,6,6,5
4,South Korea,1,1,2,1,1,4,3,6,5,2,2,4,7,3,1,1,1
5,Switzerland,2,1,5,2,1,3,6,7,1,3,6,2,6,1,1,1,5
6,Australia,4,1,3,7,1,2,4,2,4,5,3,3,2,1,1,3,5


In [12]:
# Check variable names
df.columns

Index(['country', 'pc1_dis_gfc_rank', 'pc1_dis_cpc_rank', 'pc1_dis_cov19_rank',
       'pc1_rec_gfc_rank', 'pc1_rec_cpc_rank', 'pc1_rec_cov19_rank',
       'pc2_dis_gfc_rank', 'pc2_dis_cpc_rank', 'pc2_dis_cov19_rank',
       'pc2_rec_gfc_rank', 'pc2_rec_cpc_rank', 'pc2_rec_cov19_rank',
       'cluster_shifts_flag_rank', 'policy_lag_gfc_rank',
       'policy_lag_cpc_rank', 'policy_lag_cov19_rank',
       'missing_variables_rank'],
      dtype='object')

#### Define a custom function to calculate one SRS ("Shock Resilience Score") for each country

#### Attribution: The Shock Resilience Score (SRS) model and scoring methodology were co-developed with OpenAI's ChatGPT, based on an iterative process combining human insight and AI-aided reasoning.

##### This custom function seeks to quantitatively assess and benchmark each country's economic resilience rankings on different measures through two (2) major exogenous shocks ("GFC" e.g. the Global Financial Crisis; and "Cov-19" e.g. the Covid-19 era) and one (1) moderate economic shock ("CPC" e.g. the 2014 Commodity Price Collapse).

#### Model input descriptions

##### The imported data ranking inputs to the model (custom function) were calculated in Excel. Rankings for each model input (e.g., factor) were based on the other "6.5 Task 6.5 Extension SRS" country-specific jupyter notebook PCA + k-means clustering results and exported data files, as follows:
##### Ranking for each input (factor) is on a 1 (best) to 7 (worst) integer scale, per country (there are seven (7) countries examined).
##### PC1 (Principal Component 1) and PC2 (Principal Component 2), from prior country-level PCA analysis, are significant drivers in the SRS model scoring methodology.
##### The model arrives at a single aggregated SRS (after weights are applied to each factor and shock, where applicable) for each country: the smallest aggregated raw SRS indicates best-performing country, in terms of economic resilience.
##### pc1_dis_[shock label]_rank is based on the largest magnitude change (positive or negative) in PC1 (Principal Component 1) triggered by the specified exogenous shock, from a 6-month average "baseline" figure for PC1 before the shock began, within a 24-month period from the shock start. The larger the magnitude of "disruption" compared to other countries, the worse the ranking.  There are three separate rankings on this factor for each country, corresponding to the three shocks evaluated in the model.
##### pc1_rec_[shock label]_rank is based on the extent of "recovery" of PC1 (as a %) from its prior-determined largest-magnitude change point to its greatest recovery level relative to a 20+ year simple regression line of best fit for PC1, within a 24-month period following that prior-determined largest-magnitude change point.  Comment: 20+ year regression lines for PC1 were positively-sloped trend lines for six (6) countries, except Germany, which had a negatively-sloped trend line of best fit.  Meanwhile, for PC2 (Principal Component 2), which typically accounted for mean-reversion behavior in component variables, the best fit lines were near-flat with slopes close to zero. There are three separate rankings on this factor for each country, corresponding to the three shocks evaluated in the model.
##### Ranking methodologies for the pc2 dis ("disruption") and rec ("recovery") inputs (factors) matched those used for pc1 disruption and recovery.
##### cluster_shifts_flag_rank is based on the number of cluster shifts (economic regime changes) discovered in the k-means clustering analysis for each country, across the full 20+ year temporal range of the study.  This was not unbundled further into shock-specific regime change behavior, as there were many cluster pivots that occurred outside of shock temporal periods, which should be included in the model scoring.
##### policy_lag_[shock label]_rank is based on the number of months it took a country's 10-year interest rate to meaningfully adjust to adverse inflation rate moves (or inflation rate proxies such as the Brent crude oil price (USD), if inflation rate data was lacking) during a specified exogenous crisis. This gives a rough one-dimensional idea of central bank policy responsiveness measured from a speed angle.  The Federal Reserve Bank of the United States for example, despite its presumed treasure trove and depth of available macro and microeconomic data, was slow to recognize that Covid-19 era inflation was not "transitory", causing a delayed or belated central bank response to fight inflation. Consumers in the United States are still dealing with the unintended consequences of that belated response. There are three separate rankings on this factor for each country, corresponding to the three shocks evaluated in the model.
##### missing_variables_rank is based on how many key macroeconomic variables (with data available on a monthly skew level, rather than quarterly or annual) were missing from a country-provided dataset to the OECD, with our model penalizing omissions.

#### Model weights

##### The SRS model gives equal weighting to the 1-7 ranking score of each of these seven (7) factors, for a country: PC1 disruption, PC1 recovery, PC2 disruption, PC2 recovery, number of cluster shifts (economic regime changes), policy lag (number of months it takes the 10-year interest rate to meaningfully catch up with adverse CPI moves) and number of missing key variables in a country's dataset provided the OECD.  Five of the factors are further allocated crisis-specific sub-weights as follows: GFC (Global Financial Crisis) 0.4, Covid-19 0.4 and CPC (2014 Commodities Price Collapse) 0.2. The two factors not allocated crisis-specific sub-weights are the number of cluster shifts (which is evaluated across the full 20+ year temporal study period, as more representative of that dynamic) and the number of missing variables in a country's dataset (which is crisis-agnostic).

#### Model defined

In [14]:
df.columns

Index(['country', 'pc1_dis_gfc_rank', 'pc1_dis_cpc_rank', 'pc1_dis_cov19_rank',
       'pc1_rec_gfc_rank', 'pc1_rec_cpc_rank', 'pc1_rec_cov19_rank',
       'pc2_dis_gfc_rank', 'pc2_dis_cpc_rank', 'pc2_dis_cov19_rank',
       'pc2_rec_gfc_rank', 'pc2_rec_cpc_rank', 'pc2_rec_cov19_rank',
       'cluster_shifts_flag_rank', 'policy_lag_gfc_rank',
       'policy_lag_cpc_rank', 'policy_lag_cov19_rank',
       'missing_variables_rank'],
      dtype='object')

In [16]:
# Define custom function

def compute_srs_from_ranks(
    df,
    country_col,
    shock_weights,
    pc1_dis_cols,
    pc1_rec_cols,
    pc2_dis_cols,
    pc2_rec_cols,
    policy_lag_cols,
    cluster_shift_col='cluster_shifts_flag_rank',
    missing_vars_col='missing_variables_rank',
    normalize_to_100=True
):
    scores = []

    for _, row in df.iterrows():
        total = 0
        for shock, weight in shock_weights.items():
            total += weight * (
                row[pc1_dis_cols[shock]] +
                row[pc1_rec_cols[shock]] +
                row[pc2_dis_cols[shock]] +
                row[pc2_rec_cols[shock]] +
                row[policy_lag_cols[shock]]
            )
        total += row[cluster_shift_col]
        total += row[missing_vars_col]
        scores.append((row[country_col], total))

    out = pd.DataFrame(scores, columns=[country_col, 'raw_srs'])

    if normalize_to_100:
        max_val = out['raw_srs'].max()
        min_val = out['raw_srs'].min()
        out['SRS_0_100'] = 100 * (max_val - out['raw_srs']) / (max_val - min_val)

    return out.sort_values('SRS_0_100', ascending=False).reset_index(drop=True)

#### Define the function mappings

In [19]:
shock_weights = {'gfc': 0.4, 'cov19': 0.4, 'cpc': 0.2}

pc1_dis_cols = {
    'gfc': 'pc1_dis_gfc_rank',
    'cpc': 'pc1_dis_cpc_rank',
    'cov19': 'pc1_dis_cov19_rank'
}
pc1_rec_cols = {
    'gfc': 'pc1_rec_gfc_rank',
    'cpc': 'pc1_rec_cpc_rank',
    'cov19': 'pc1_rec_cov19_rank'
}
pc2_dis_cols = {
    'gfc': 'pc2_dis_gfc_rank',
    'cpc': 'pc2_dis_cpc_rank',
    'cov19': 'pc2_dis_cov19_rank'
}
pc2_rec_cols = {
    'gfc': 'pc2_rec_gfc_rank',
    'cpc': 'pc2_rec_cpc_rank',
    'cov19': 'pc2_rec_cov19_rank'
}
policy_lag_cols = {
    'gfc': 'policy_lag_gfc_rank',
    'cpc': 'policy_lag_cpc_rank',
    'cov19': 'policy_lag_cov19_rank'
}

#### Call the compute_srs_from_ranks custom function

In [22]:
srs_df = compute_srs_from_ranks(
    df=df,  # your DataFrame
    country_col='country',
    shock_weights=shock_weights,
    pc1_dis_cols=pc1_dis_cols,
    pc1_rec_cols=pc1_rec_cols,
    pc2_dis_cols=pc2_dis_cols,
    pc2_rec_cols=pc2_rec_cols,
    policy_lag_cols=policy_lag_cols,
    cluster_shift_col='cluster_shifts_flag_rank',
    missing_vars_col='missing_variables_rank',
    normalize_to_100=True
)

#### Output the created SRS dataframe

In [25]:
print(srs_df)

         country  raw_srs   SRS_0_100
0         Norway     18.8  100.000000
1    South Korea     20.6   86.764706
2      Australia     23.0   69.117647
3    Switzerland     24.6   57.352941
4        Germany     28.4   29.411765
5          Japan     29.4   22.058824
6  United States     32.4    0.000000


##### The higher the SRS-normalized-to-100 score, the more resilient the economy. The three (3) countries with the largest populations and GDP within the group of seven (7) countries scored the worst.  Population and GDP size do not appear to influence economic resilience positively at all.

In [27]:
# Export the SRS dataframe
path = r'C:\Users\dirk8\CareerFoundry Projects\05-2025 Exogenous Shocks Analysis'
srs_df.to_excel(os.path.join(path, 'Data', 'Prepared Data', 'SRS Final Scores.xlsx'))

### 03. Key project finding

## Norway is the most resilient country, from a macroeconomic perspective, over the last 20+ years and through multiple exogenous economic shocks.  The United States is the least resilient country of the seven (7) OECD members examined, over the same period, using our custom-defined SRS model and based on data observations throughout this ESA ("Exogenous Shocks Analysis") project.

## If economic regime stability through frequent exogenous shocks is a priority for a multinational corporation, it may wish to consider many countries beyond the United States as economic safe haven candidates.

### Archived

#### Memo item: version of the custom function allowing customization of specific weights for the model inputs.

In [62]:
import pandas as pd

def compute_srs_from_ranks_custom_weights(
    df,
    country_col,
    shock_weights,
    metric_weights,
    pc1_dis_cols,
    pc1_rec_cols,
    pc2_dis_cols,
    pc2_rec_cols,
    policy_lag_cols,
    cluster_shift_col='cluster_shifts_flag_rank',
    cluster_shift_weight=1.0,
    missing_vars_col='missing_variables_rank',
    missing_vars_weight=1.0,
    normalize_to_100=True
):
    scores = []

    for _, row in df.iterrows():
        total = 0
        for shock, shock_wt in shock_weights.items():
            total += shock_wt * (
                metric_weights['pc1_dis'] * row[pc1_dis_cols[shock]] +
                metric_weights['pc1_rec'] * row[pc1_rec_cols[shock]] +
                metric_weights['pc2_dis'] * row[pc2_dis_cols[shock]] +
                metric_weights['pc2_rec'] * row[pc2_rec_cols[shock]] +
                metric_weights['policy_lag'] * row[policy_lag_cols[shock]]
            )
        total += cluster_shift_weight * row[cluster_shift_col]
        total += missing_vars_weight * row[missing_vars_col]
        scores.append((row[country_col], total))

    out = pd.DataFrame(scores, columns=[country_col, 'raw_srs'])

    if normalize_to_100:
        max_val = out['raw_srs'].max()
        min_val = out['raw_srs'].min()
        out['SRS_0_100'] = 100 * (max_val - out['raw_srs']) / (max_val - min_val)

    return out.sort_values('SRS_0_100', ascending=False).reset_index(drop=True)